In [1]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import copy
import functools
from importlib import reload
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow import keras

from src.harness import constants as C
from src.harness import dataset as ds
from src.harness import history
from src.harness import experiment
from src.harness import mixins
from src.harness import model as mod
from src.harness import paths
from src.harness import pruning
from src.harness import rewind
from src.harness import training as train
from src.harness import utils

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Parameters

In [3]:
reload(ds)
reload(mod)
reload(pruning)

# Select the dataset
mnist_dataset: ds.Dataset = ds.Dataset(ds.Datasets.MNIST)
X_train, X_test, Y_train, Y_test = mnist_dataset.load()
input_shape: tuple = mnist_dataset.input_shape
num_classes: int = mnist_dataset.num_classes

print(f'Input Shape: {input_shape}')
print(f'Num Classes: {num_classes}')
print(f'X_train Shape: {X_train.shape}, Y_train Shape: {Y_train.shape}')
print(f'X_test Shape: {X_test.shape}, Y_test Shape: {Y_test.shape}')

num_epochs: int = 10
batch_size: int = len(X_train)

Input Shape: (1, 784)
Num Classes: 10
X_train Shape: (60000, 1, 784), Y_train Shape: (60000, 1, 10)
X_test Shape: (10000, 1, 784), Y_test Shape: (10000, 1, 10)


## Building

In [4]:
reload(ds)
reload(mod)
reload(utils)

# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
utils.set_seed(0)
make_lenet: callable = functools.partial(mod.create_lenet_300_100, input_shape, num_classes)

original_model: keras.Model = make_lenet()
# original_model.summary()
# original_model.trainable_variables

original_mask_model: keras.Model = mod.create_masked_nn(make_lenet)
original_mask_model.summary()
# original_mask_model.trainable_variables

Model: "LeNet-300-100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 300)            235500    
                                                                 
 dense_4 (Dense)             (None, 1, 100)            30100     
                                                                 
 dense_5 (Dense)             (None, 1, 10)             1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


## Training

In [5]:
reload(C)
reload(train)

# Use the original model as a reference
loss_fn: tf.keras.losses.Loss = C.LOSS_FUNCTION()
accuracy_metric: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

test_loss, test_accuracy = train.test_step(original_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

Test Loss: 2.368942, Test Accuracy: 0.094900


2024-04-28 14:59:28.644431: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
reload(train)

# Test that deepcopy is exactly the same as the original model
# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)

test_loss, test_accuracy = train.test_step(original_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

Test Loss: 2.368942, Test Accuracy: 0.094900


In [7]:
reload(C)
reload(train)

# Test single step of training

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()
train_one_step: callable = train.get_train_one_step()
accuracy_metric.reset_states()

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)

# Sanity Check
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

epochs: int = 5
batch_size: int = len(X_train)

train_accuracies: np.array = np.zeros(epochs)
test_accuracies: np.array = np.zeros(epochs)

for i in range(epochs):
    print(f'Starting Epoch {i + 1}')
    for n in range(int(np.ceil(len(X_train) / batch_size))):
        train_loss, train_accuracy = train_one_step(model, mask_model, X_train[n * batch_size:(n + 1) * batch_size], Y_train[n * batch_size:(n + 1) * batch_size], optimizer)
        train_accuracies[i] = train_accuracy

        test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
        test_accuracies[i] = test_accuracy

        print(f'Batch {n + 1} Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.6f}, Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

print(f'Test Accuracies:')
print(test_accuracies)
print(f'Training Accuracies:')
print(train_accuracies)

# Get test parameters
accuracy_metric.reset_states()
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')


Test Loss: 2.368942, Test Accuracy: 0.094900

Starting Epoch 1
Batch 1 Train Loss: 2.358066, Train Accuracy: 0.105133, Test Loss: 1.731715, Test Accuracy: 0.461700
Starting Epoch 2
Batch 1 Train Loss: 1.736589, Train Accuracy: 0.466583, Test Loss: 1.204787, Test Accuracy: 0.693200
Starting Epoch 3
Batch 1 Train Loss: 1.223354, Train Accuracy: 0.691967, Test Loss: 0.846864, Test Accuracy: 0.764500
Starting Epoch 4
Batch 1 Train Loss: 0.871248, Train Accuracy: 0.755833, Test Loss: 0.623985, Test Accuracy: 0.809200
Starting Epoch 5
Batch 1 Train Loss: 0.648661, Train Accuracy: 0.795700, Test Loss: 0.538173, Test Accuracy: 0.824500
Test Accuracies:
[0.46169999 0.69319999 0.76450002 0.80919999 0.82450002]
Training Accuracies:
[0.10513333 0.46658334 0.69196665 0.75583333 0.79570001]

Test Loss: 0.538173, Test Accuracy: 0.824500


In [8]:
reload(C)
reload(ds)
reload(train)

# Testing `training_loop` function
epochs: int = C.TRAINING_EPOCHS

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)

# Sanity Check
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

trial_data = train.training_loop(0, model, mask_model, mnist_dataset, epochs)

print(f'Took {np.sum(trial_data.test_accuracies != 0)} / {epochs} epochs')
print(f'Ended with a best training accuracy of {np.max(trial_data.train_accuracies) * 100:.2f}% and test accuracy of {np.max(trial_data.test_accuracies) * 100:.2f}%')

print(f'Test Accuracies:')
print(trial_data.test_accuracies)
print(f'Training Accuracies:')
print(trial_data.train_accuracies)

# Get test parameters
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')


Test Loss: 2.368942, Test Accuracy: 0.094900

Step 0 of Iterative Magnitude Pruning


/Users/jordan/anaconda3/envs/lottery-tickets/lib/python3.11/site-packages/numpy/lib/function_base.py:959: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Epoch 1 Train Loss: 0.231, Train Accuracy: 0.929, Test Loss: 0.154, Test Accuracy: 0.953
Epoch 2 Train Loss: 0.105, Train Accuracy: 0.967, Test Loss: 0.148, Test Accuracy: 0.957
Epoch 3 Train Loss: 0.075, Train Accuracy: 0.977, Test Loss: 0.121, Test Accuracy: 0.967
Epoch 4 Train Loss: 0.059, Train Accuracy: 0.981, Test Loss: 0.169, Test Accuracy: 0.957
Epoch 5 Train Loss: 0.054, Train Accuracy: 0.983, Test Loss: 0.122, Test Accuracy: 0.967
Epoch 6 Train Loss: 0.051, Train Accuracy: 0.983, Test Loss: 0.121, Test Accuracy: 0.969
Epoch 7 Train Loss: 0.046, Train Accuracy: 0.986, Test Loss: 0.120, Test Accuracy: 0.974
Epoch 8 Train Loss: 0.040, Train Accuracy: 0.987, Test Loss: 0.135, Test Accuracy: 0.973
Epoch 9 Train Loss: 0.038, Train Accuracy: 0.988, Test Loss: 0.137, Test Accuracy: 0.973
Epoch 10 Train Loss: 0.040, Train Accuracy: 0.988, Test Loss: 0.114, Test Accuracy: 0.978
Epoch 11 Train Loss: 0.030, Train Accuracy: 0.990, Test Loss: 0.134, Test Accuracy: 0.976
Epoch 12 Train Loss

In [10]:
# Testing `train` function

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)

# Sanity Check
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

trial_data = train.train(0, 0, model, mask_model, mnist_dataset, batch_size=C.BATCH_SIZE)

print(f'\nTook {np.sum(trial_data.test_accuracies != 0)} / {C.TRAINING_EPOCHS} epochs')
print(f'Ended with a best training accuracy of {np.max(trial_data.train_accuracies) * 100:.2f}% and test accuracy of training accuracy of {np.max(trial_data.test_accuracies) * 100:.2f}%')

print(f'Test Accuracies:')
print(trial_data.test_accuracies)
print(f'Training Accuracies:')
print(trial_data.train_accuracies)

# Get test parameters
test_loss, test_accuracy = train.test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')


Test Loss: 2.368942, Test Accuracy: 0.094900

Step 0 of Iterative Magnitude Pruning
Epoch 1 Train Loss: 0.286, Train Accuracy: 0.917, Test Loss: 0.146, Test Accuracy: 0.957
Epoch 2 Train Loss: 0.117, Train Accuracy: 0.964, Test Loss: 0.139, Test Accuracy: 0.960
Epoch 3 Train Loss: 0.082, Train Accuracy: 0.974, Test Loss: 0.127, Test Accuracy: 0.964
Epoch 4 Train Loss: 0.065, Train Accuracy: 0.979, Test Loss: 0.114, Test Accuracy: 0.969
Epoch 5 Train Loss: 0.053, Train Accuracy: 0.984, Test Loss: 0.121, Test Accuracy: 0.969
Epoch 6 Train Loss: 0.047, Train Accuracy: 0.985, Test Loss: 0.122, Test Accuracy: 0.970
Epoch 7 Train Loss: 0.038, Train Accuracy: 0.987, Test Loss: 0.126, Test Accuracy: 0.969
Epoch 8 Train Loss: 0.039, Train Accuracy: 0.988, Test Loss: 0.144, Test Accuracy: 0.969
Epoch 9 Train Loss: 0.039, Train Accuracy: 0.987, Test Loss: 0.141, Test Accuracy: 0.967
Epoch 10 Train Loss: 0.033, Train Accuracy: 0.989, Test Loss: 0.151, Test Accuracy: 0.971
Early stopping initiated

In [11]:
# Test loading the model back
loaded_model: keras.Model = mod.load_model(0, 0)

# Get test parameters
test_loss, test_accuracy = train.test_step(loaded_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')


Test Loss: 0.150861, Test Accuracy: 0.971200


In [12]:
print(f'Nonzero parameters after training but before pruning:')
print(utils.count_total_and_nonzero_params(model)[1])

Nonzero parameters after training but before pruning:
266610


## Pruning

In [13]:
# Test loading the model back
loaded_model: keras.Model = mod.load_model(0, 0)
target_sparsity = 0.5

# Get test parameters
test_loss, test_accuracy = train.test_step(loaded_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

sparse_model = copy.deepcopy(loaded_model)

total, nonzero = utils.count_total_and_nonzero_params(loaded_model)
print(f'Total Params: {total}, Nonzero Params: {nonzero}')
print('Pruning')
pruning.prune(sparse_model, pruning.low_magnitude_pruning, target_sparsity)

total2, nonzero2 = utils.count_total_and_nonzero_params(sparse_model)
print(f'Total Params: {total}, Nonzero Params: {nonzero}')

assert total2 == total
assert nonzero2 == nonzero // 2


Test Loss: 0.150861, Test Accuracy: 0.971200
Total Params: 266610, Nonzero Params: 266610
Pruning
Total Params: 266610, Nonzero Params: 266610


In [14]:
reload(mod)
reload(rewind)
    
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)
original_weights = model.get_weights()

rewind_to_original_weights: callable = functools.partial(rewind.rewind_to_original_init, 0)
rewind_to_random_weights: callable = functools.partial(rewind.rewind_to_random_init, 0, tf.initializers.GlorotUniform())
rewind.rewind_model_weights(model, mask_model, rewind_to_random_weights)

print(original_weights[0][0][:10])
print(model.get_weights()[0][0][:10])

[ 0.00233377 -0.02841079  0.05042759 -0.05853604 -0.01563397 -0.04661581
 -0.00013836  0.07242373  0.00807343 -0.00930607]
[ 0.00233377 -0.02841079  0.05042759 -0.05853604 -0.01563397 -0.04661581
 -0.00013836  0.07242373  0.00807343 -0.00930607]


/Users/jordan/anaconda3/envs/lottery-tickets/lib/python3.11/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [15]:
reload(mod)
reload(pruning)

model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(original_mask_model)

# Asserting that every array in the mask model's weights are 1s
for layer in mask_model.layers:
    for weights in layer.get_weights():
        assert np.all(weights == 1), "Error: Not all elements in mask model's weights are 1s after updating masks"

pruning.update_masks(model, mask_model)

# Asserting that every array in the mask model's weights are still 1s
for layer in mask_model.layers:
    for weights in layer.get_weights():
        assert np.all(weights == 1), "Error: Not all elements in mask model's weights are 1s after updating masks"
        
pruning.prune(model, pruning.low_magnitude_pruning, 0.5)

## Experiments

In [16]:
reload(experiment)
reload(pruning)
reload(rewind)
reload(train)

# Pruning Parameters
first_step_pruning: float = 0.2
target_sparsity: float = 0.79
make_lenet: callable = functools.partial(mod.create_lenet_300_100, input_shape, num_classes)

global_pruning: bool = False
sparsities: list[float] = pruning.get_sparsity_percents(model, first_step_pruning, target_sparsity)
experiment_data: history.ExperimentData = experiment.run_iterative_pruning_experiment(
    0, 
    make_lenet, 
    mnist_dataset,
    sparsities,
)

Directory 'models/model_0/trial0/weights' already exists.
Directory 'models/model_0/trial0/masks' already exists.
Step 0 of Iterative Magnitude Pruning
Epoch 1 Train Loss: 0.231, Train Accuracy: 0.929, Test Loss: 0.154, Test Accuracy: 0.953
Epoch 2 Train Loss: 0.105, Train Accuracy: 0.967, Test Loss: 0.148, Test Accuracy: 0.957
Epoch 3 Train Loss: 0.075, Train Accuracy: 0.977, Test Loss: 0.121, Test Accuracy: 0.967
Epoch 4 Train Loss: 0.059, Train Accuracy: 0.981, Test Loss: 0.169, Test Accuracy: 0.957
Epoch 5 Train Loss: 0.054, Train Accuracy: 0.983, Test Loss: 0.122, Test Accuracy: 0.967
Epoch 6 Train Loss: 0.051, Train Accuracy: 0.983, Test Loss: 0.121, Test Accuracy: 0.969
Epoch 7 Train Loss: 0.046, Train Accuracy: 0.986, Test Loss: 0.120, Test Accuracy: 0.974
Epoch 8 Train Loss: 0.040, Train Accuracy: 0.987, Test Loss: 0.135, Test Accuracy: 0.973
Epoch 9 Train Loss: 0.038, Train Accuracy: 0.988, Test Loss: 0.137, Test Accuracy: 0.973
Epoch 10 Train Loss: 0.040, Train Accuracy: 0.9

In [17]:
reload(experiment)
reload(train)
reload(utils)

def to_str(round: history.TrialData) -> str:
    representation: str = f"""Pruning Step {round.pruning_step}
    Sparsity: {round.get_sparsity() * 100:.3f}%
    Best Training Accuracy: {round.get_best_accuracy(use_test=False) * 100:.3f}%
    Best Test Accuracy: {round.get_best_accuracy() * 100:.3f}%
    Best Training Loss: {round.get_best_loss(use_test=False):.3f}%
    Best Test Loss: {round.get_best_loss():.3f}%
    """
    return representation

for step_index in range(len(sparsities)):
    round = experiment_data.pruning_rounds[step_index]
    print(round)

Pruning Step 0
        Sparsity: 100.000%
        Best Training Accuracy: 99.238%
        Best Test Accuracy: 97.850%
        Best Training Loss: 0.231
        Best Test Loss: 0.169
        
Pruning Step 1
        Sparsity: 80.025%
        Best Training Accuracy: 99.058%
        Best Test Accuracy: 97.430%
        Best Training Loss: 0.233
        Best Test Loss: 0.167
        
Pruning Step 2
        Sparsity: 64.023%
        Best Training Accuracy: 99.093%
        Best Test Accuracy: 97.660%
        Best Training Loss: 0.225
        Best Test Loss: 0.167
        


In [18]:
reload(experiment)
reload(paths)
reload(pruning)
reload(rewind)
reload(train)

# Test high level experiment API
experiment_directory: str = 'testing_experiment'
experiment_summary: history.ExperimentSummary = experiment.run_experiments(
    2, 
    experiment_directory,
    functools.partial(experiment.get_mnist_lenet_300_100_experiment_parameters, 0.2, 0.65),
    experiment.run_iterative_pruning_experiment,
)

Directory 'testing_experiment' created successfully.
Directory 'models/model_0/trial0/weights' already exists.
Directory 'models/model_0/trial0/masks' already exists.
Step 0 of Iterative Magnitude Pruning
Epoch 1 Train Loss: 0.231, Train Accuracy: 0.929, Test Loss: 0.154, Test Accuracy: 0.953
Epoch 2 Train Loss: 0.105, Train Accuracy: 0.967, Test Loss: 0.148, Test Accuracy: 0.957
Epoch 3 Train Loss: 0.075, Train Accuracy: 0.977, Test Loss: 0.121, Test Accuracy: 0.967
Epoch 4 Train Loss: 0.059, Train Accuracy: 0.981, Test Loss: 0.169, Test Accuracy: 0.957
Epoch 5 Train Loss: 0.054, Train Accuracy: 0.983, Test Loss: 0.122, Test Accuracy: 0.967
Epoch 6 Train Loss: 0.051, Train Accuracy: 0.983, Test Loss: 0.121, Test Accuracy: 0.969
Epoch 7 Train Loss: 0.046, Train Accuracy: 0.986, Test Loss: 0.120, Test Accuracy: 0.974
Epoch 8 Train Loss: 0.040, Train Accuracy: 0.987, Test Loss: 0.135, Test Accuracy: 0.973
Epoch 9 Train Loss: 0.038, Train Accuracy: 0.988, Test Loss: 0.137, Test Accuracy: 

In [19]:
for seed, experiment_data in experiment_summary.experiments.items():
    print(f'Model {seed}')
    for round in experiment_data.pruning_rounds:
        print(round)

Model 0
Pruning Step 0
        Sparsity: 100.000%
        Best Training Accuracy: 99.238%
        Best Test Accuracy: 97.850%
        Best Training Loss: 0.231
        Best Test Loss: 0.169
        
Pruning Step 1
        Sparsity: 80.025%
        Best Training Accuracy: 99.058%
        Best Test Accuracy: 97.430%
        Best Training Loss: 0.233
        Best Test Loss: 0.167
        
Pruning Step 2
        Sparsity: 64.023%
        Best Training Accuracy: 99.093%
        Best Test Accuracy: 97.660%
        Best Training Loss: 0.225
        Best Test Loss: 0.167
        
Pruning Step 0
        Sparsity: 100.000%
        Best Training Accuracy: 99.238%
        Best Test Accuracy: 97.850%
        Best Training Loss: 0.231
        Best Test Loss: 0.169
        
Pruning Step 1
        Sparsity: 80.025%
        Best Training Accuracy: 99.058%
        Best Test Accuracy: 97.430%
        Best Training Loss: 0.233
        Best Test Loss: 0.167
        
Pruning Step 2
        Sparsity: 64.023%

In [24]:
loaded_experiment_summary = history.ExperimentData.load_from(os.path.join('testing_experiment', 'experiment_summary.pkl'))
for seed, experiment_data in loaded_experiment_summary.experiments.items():
    print(f'Model {seed}')
    for round in experiment_data.pruning_rounds:
        print(round) 

Model 0
Pruning Step 0
        Sparsity: 100.000%
        Best Training Accuracy: 99.238%
        Best Test Accuracy: 97.850%
        Best Training Loss: 0.231
        Best Test Loss: 0.169
        
Pruning Step 1
        Sparsity: 80.025%
        Best Training Accuracy: 99.058%
        Best Test Accuracy: 97.430%
        Best Training Loss: 0.233
        Best Test Loss: 0.167
        
Pruning Step 2
        Sparsity: 64.023%
        Best Training Accuracy: 99.093%
        Best Test Accuracy: 97.660%
        Best Training Loss: 0.225
        Best Test Loss: 0.167
        
Pruning Step 0
        Sparsity: 100.000%
        Best Training Accuracy: 99.238%
        Best Test Accuracy: 97.850%
        Best Training Loss: 0.231
        Best Test Loss: 0.169
        
Pruning Step 1
        Sparsity: 80.025%
        Best Training Accuracy: 99.058%
        Best Test Accuracy: 97.430%
        Best Training Loss: 0.233
        Best Test Loss: 0.167
        
Pruning Step 2
        Sparsity: 64.023%